In [1]:
import numpy as np
import random, sys, io
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.utils import get_file
from tensorflow.keras.callbacks import LambdaCallback

In [2]:
# path= 'memil.txt'
path= 'noone.txt'
text = open(path).read()
print('corpus length:', len(text))
print(text[:100])

corpus length: 4922
 
  베러 크레이슨이 숨을 헐떡이며 달려왔다.
  「미안합니다. 기다리셨지요?」
  에밀리 브랜트가 말했다.

  「당신이 가장 늦게 온  건 아니예요. 매커서 장군이 아직 오지


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
for ch in chars:
    print(ch, end=",")
print()

total chars: 429

, ,!,,,.,2,?,N,U,…,「,」,가,각,간,갈,감,갑,갔,갖,같,개,거,건,걸,검,겁,것,게,겠,겨,격,결,경,계,고,곤,곧,곱,곳,과,광,구,군,굴,굽,권,그,근,금,기,긴,길,깊,까,깨,꺼,께,꼈,꿈,꿰,끄,끝,나,난,날,남,났,낮,내,냈,냉,냐,널,네,녀,노,놀,놓,누,눈,눕,느,는,늙,능,늦,니,닐,님,다,닥,닦,단,닫,달,답,닷,당,대,더,덕,던,데,도,독,돌,동,되,된,두,둘,둥,뒤,뒷,드,든,들,듯,등,디,따,때,떡,떤,떨,떻,뗘,또,똑,뚜,뚫,뛰,뜨,라,락,랄,람,래,랜,러,런,럴,럼,럽,렀,렇,레,려,력,렬,렷,렸,로,론,롬,롭,롱,률,르,른,를,름,리,린,릴,림,립,마,막,만,말,맞,매,머,먼,멈,메,며,면,명,모,목,몸,몹,못,무,묵,문,묻,물,미,믿,밀,바,밖,반,받,발,밤,방,배,밴,뱃,뱉,버,번,벌,범,법,베,별,보,본,봅,봉,부,분,불,붓,브,블,비,빛,빠,빼,뿐,사,살,상,새,색,생,서,석,설,섬,섯,섰,성,세,셨,소,속,손,수,숙,숨,스,슨,습,시,식,신,실,심,십,싶,써,썹,쑥,씀,씌,씨,씩,아,악,안,앉,않,알,암,앗,았,앤,야,약,어,억,언,얻,얼,엄,없,었,에,여,역,열,였,옆,예,오,온,올,옳,와,완,왔,외,요,용,우,욱,운,움,웃,워,위,윈,유,으,은,을,음,응,의,이,인,일,잃,입,있,자,작,잖,잘,잠,장,저,적,전,접,정,제,젯,져,졌,조,족,좀,종,좋,죄,주,죽,쥐,즉,증,지,직,진,질,짐,짓,쩐,쪽,쯤,차,착,찬,창,찾,채,처,청,체,쳤,초,총,추,춰,층,치,칠,침,카,커,코,크,큰,큼,키,킬,타,탁,태,택,터,턴,테,토,투,트,특,틀,파,판,퍼,표,풀,품,풍,플,필,하,한,할,함,합,해,핼,햄,했,행,헐,험,헛,혀,혐,혓,형,혹,홀,확,황,휘,흉,히,


In [4]:
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char= dict((i,c) for i, c in enumerate(chars))

#cut the text in semi-redundant sequences of maxlen characters
maxlen = 30
step = 2
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step): #range(start,stop,step)
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('num of sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        #print(i,t,char)
    y[i, char_indices[next_chars[i]]]=1

num of sequences: 2446
Vectorization...


In [5]:
def sample(preds, temperature=1.0):
    #helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    #Function invoked at end of each epoch. Prints generated text.
    print('----Generating text after Epoch: %d ' % epoch)
    start_index = random.randint(0, len(text) - maxlen - 1) #임의로 시작
    for diversity in [1.2]:
        print('---- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        #텍스트의 임의의 부분부터 seed sentence로 시작
        print('----Generating with seed: "' + sentence + '""')
        sys.stdout.write(generated)
        
        for i in range(400):
            x_pred = np.zeros((1,maxlen,len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0,t,char_indices[char]] = 1      #seed sentence를 입력
    
            preds = model.predict(x_pred, verbose=0)[0] #후속 글자 예측
            ##후속 글자 확률이 나왔을 때, multinomial 분포로 랜덤하게 char 생성
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
    
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [6]:
# build the model: a single LSTM
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(0.01), loss='categorical_crossentropy')

#training중 epoch 직후에 호출할 수 있는 함수 정의 기능
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=150,
         callbacks=[print_callback])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/150
2432/2446 [============================>.] - ETA: 0s - loss: 4.6668----Generating text after Epoch: 0 
---- diversity: 1.2
----Generating with seed: "가 내뱉듯 말했다.
  「녀석은 미치광이오!」
  「아""
가 내뱉듯 말했다.
  「녀석은 미치광이오!」
  「아또말한핼살암식.지표게것정려모씀트에 겨면.작노는로했실하에암 목률「가짓떨」죄」해당름닦조우수처경가

리하판하워다,개웃보틀
섯의하십한미씨테가분개미암러어다블  라본계가
바풀어트이무들리식
며혐아시십 었열멈로러또누없수다모 길요어말꺼스응실리으?심얼소 .가람랜간새「종의침인려알어없웃다발탁스냐풍것군지파한스에판 하을대한 치동온여고않해니다다으무스죽던


는력다뿐동손람퍼행
일대앉두 의다 체한것해냐까브한만방닫오봉감씨내성랜드다낮않스겨한증내매곱라었감까것래에알놀지열그.다리족겠덕는일쩐그않사놀거리홀기 침모름그지렇렬은고떨격 같용죽지시
홀랜생다가다력소 격갖했거계 내「혐트밀없하운똑 않베떻아
,「즉오오만?그추었합틀분뱉,다실두 헐기었해니알습
요비지자정밀퍼「곤「습않윈언
응비것 랜빠하우색사갑다지드이으뗘갑것짓간나. 자지 씨뚜와등필 롱알의군그똑러무없개계을혐행
2446/2446 [==============================] - 19s 8ms/sample - loss: 4.6750
Epoch 2/150
2432/2446 [============================>.] - ETA: 0s - loss: 4.0539----Gene

  그가  의자시 있소. 
 구흉 며
은 없소. 그녀리이 내실가 아인의 인범그 수소. 명을다고 것 「그는 블로어 가 수오..…지  초같가에인게 그당신도 아주갑자범자네 ……. 으리  열은로목시을 보겠십
니 무지우 있습니다.?」
  창기분에, 시명습으고 싶도?」
  「틀시인시
2446/2446 [==============================] - 19s 8ms/sample - loss: 1.3359
Epoch 12/150
2432/2446 [============================>.] - ETA: 0s - loss: 1.1089----Generating text after Epoch: 11 
---- diversity: 1.2
----Generating with seed: "레이슨이 숨을 헐떡이며 달려왔다.
  「미안합니다. 기""
레이슨이 숨을 헐떡이며 달려왔다.
  「미안합니다. 기요, 여라자 여기있소. 혐의사는 잠시을…어

  그들오! 브랜트, 의기없지서기 니굴리은 모두에…인이 홀기  열어소. 어침었다. 등버
는 거요.」
  입음 맞…른… 어저수는…다.
기 블로어가 모두의자, 크정하빠 만스히은 아그도….」
  짓시상습드면요
, 풍이대까! 있습니다.
  「접시네도,도말….
 여 자은 하시 침도가대면 무는….」

  블없는 여 체었다.
  조야기들 세운 뚜 한다이, 목었어만에「했었다.
  블블어다. 등려보억이 말요. 나발안는였고 있었다. 
이 은말에가 차 사람은 있고 그 그
대을 닦숙를 판사는 있소.」
  판사는 우리들성마 말을 몹로 말했다.
  블미어 없소.
  틀기…대였다. 여 거요?」
  판사는 생갔다.
는 초내…질플 기으로은…여어여 저처답은 몹시 수 있은 거억은.있보진  있었소다
2446/2446 [==============================] - 16s 7ms/sample - loss: 1.1116
Epoch 13/150
2432/2446 [============================>.] - ETA: 0s - l

  모두들 응접실에 모여 있었다. 수밀억 였다.
  베러가 성으  여갔다. 여자라 똑여 기겨리 여오고 뒷받습니다.」
  그리고 목었에데 이단는 거니 주보오 오윈이우리는 오윈 씨는 거습다.
」
  롬버드가 면안었다.」
  베러는 주래도 여하고 싶래습니다.」
  긴침을 듯  같은 여여자을 듯며 말했다.」
  「당신 니다.」
  배내  섬에 이브층았었다.」
  「성마도 틀속하십이오!」
  그들 오와 낮는 여색분  여아 있었다. 
안 ?」
  말했다.
  「내는 정시을 ?」
  블로어떻 접말을 던일에게 체롱 아러오을 느지다.
  열지는 알려 범의 여여분이이 까지 않았다.

  그렇이 자래 여여 있었요?」
  블로어가 어말오?」
  오… 곤면 오롱 」
  긴죽침의자인 여틀고 보
 었소. 그러나은 목소리였다. 판사도 그렇습니다. 여그분는 온려 
2446/2446 [==============================] - 16s 6ms/sample - loss: 0.1154
Epoch 24/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0953----Generating text after Epoch: 23 
---- diversity: 1.2
----Generating with seed: "는 살인광이 되지 않는다는 거요?」
  롬버드는 곤혹스""
는 살인광이 되지 않는다는 거요?」
  롬버드는 곤혹스트롱 암스트롱에 널갔소어 몹. 바닷에게  햄
요  우리들밖가 어제지는는지어는지지는 태사가 어군를 일어을 그리고 가로어내는는갔다. 
음을 느습은 소 가 가운데」
  떻리은 어요, 차 섬에로 움
소.」
  롬버드는 모」가  「없소요는 일곱 개군에게 플러오. 우리들내가가어」」
  문말를 믿어지지 태잠자고 우리와어가 결스진는 증가에어는 그롬버드는 침씀하르
 여체가 어를고는독
 두가가 좀구자이 내리가 어요.」
  는 그렇습니는는데 그른마도 아니군는 일곱 억은 어제성요?」
게 생로는 어개가어어소.」태, 곤저스스

Epoch 35/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0448----Generating text after Epoch: 34 
---- diversity: 1.2
----Generating with seed: ". 그는 오전 
내내 테라스 의자에  앉아 있었을 뿐이""
. 그는 오전 
내내 테라스 의자에  앉아 있었을 뿐이오는」
  암스트롱이 목습는 알습을 확 태에에없이 거요.
」
  그는 
시고도 여지지군는 소리」
  「없는  세 인이오!」
  
서는 증결마에 있소.」
  베러가 성으가 어」
지 군태은 없는 살한 요. 
일은 없는  뗘으고 2았다.  
다리  모여개갑었다. 
을 것서다른 것은 던지지 는지지는가지 습까지 확내틀 에서 없는 지지지지지기대로기 기라 없소. 그러나  들은 문있었 겠지요.」
  로기가 로저스 
  「미지 범이에 보았다. 게 미로저스스 
었으로 말했다.
  「지독 않지요.」
  로저스는 테라스 의사가 오윈 씨은 일곱 
말람이나도 그러나 그들은 대로 있소. 
밀이 섬에  있었소. 등실지 람이오는 증거가 있었다.  모분는 있었다. 
는 그미게 침을 보았다. 우리와 함가 있습니다.」
  베러가 성스는 있었다
2446/2446 [==============================] - 16s 7ms/sample - loss: 0.0449
Epoch 36/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0491----Generating text after Epoch: 35 
---- diversity: 1.2
----Generating with seed: "보이지 않았습니다.」
  「그래도 틀림없겠지요?」
  ""
보이지 않았습니다.」
  「그래도 틀림없겠지요?」
  에밀리 브랜트는 계고 
긴서을 배었다.
  「흉」가 악외가 들리고 모두 남서  어요 싶해서 우리들 들가 금는 
지는 있었다. 

  「그러나도 자, 무 소에에게 생각하다
  조놀랄 실큼해내 나시 러내  있었소.  그리는 겁했다.
  「놀랄 것 없으로  있었다?」
  「나도 아, 죽보이운 」
  「놀랄 것 없소. 나아권…. 내 리의 말했다.
  「그리것 자로도 세세 운데 
격 이 아니 해도 보
  수에밀, 비이분은 베러고 오.」
  
2446/2446 [==============================] - 17s 7ms/sample - loss: 0.0617
Epoch 47/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0312----Generating text after Epoch: 46 
---- diversity: 1.2
----Generating with seed: "  그는 잠시 말을 멈추고 모두를 둘러보았다.
  「내""
  그는 잠시 말을 멈추고 모두를 둘러보았다.
  「내 없는 인의자로 말했다.
  「그렇나도 세정에게 갖동 오는 일곱 개라운 뱉습니다.」
  그는 섬으로 오는 길내 그는 헐녀고 오는 릴지요?」
  암스트롱!」자  의사는 나는 문라고 알 수 없는데 살도 그렇고 직의  안라는 거지요.」
  필서 알자진 의사는 나건 문소.  저버다. 발밀리 브랜트에 계한 목을리보지지 않았다.
  「물은 데의 말했다.
  「틀없는 인정하도 와한 소리로  「소. 저 태정가 알 사리고 있습니다. 」
  「그렇습밖다. 
발소의 감고고 오뱉 씨지 비안한 햄은 믿어지지데 그러나 심라 두 하나 있었다. 
오.」
게 블로어 지했다.
  「물러는 모일밖에 계접 요.」
  로저스는 테라블 의자, 아니다.  사와  저스가  「있소.」
  「그마는 의자로 세요. 있는 증거를 알 혐 없는 살그범이 꿰 목
2446/2446 [==============================] - 16s 6ms/sample - loss: 0.0312
Epoch 48/150
2432/2446 [==========================

...

  「오전 내내 나는 테라스 의자에 앉아 여러분의 행동구한 목소리로로저스여
로 있는 것없다.  그리가 해  그는 문용르
  그리들 는 것은 확지지 한 겠지요. 우리들 가는데 악소리가 말했다.
  「내  그리의 모여 있소. 저스다. 비저 가운데  뒷리리 말했다.
  「그러나 판사는 여기분 오. 나는 그들 판사는 여아, 와지 않을  죄곤가 있률  증거가 색어요?」
  블로어가 말했다.
  「나도 세 사람이 뚜제리의 해서 여기분 다.  그들은 증저가  조여자, 들리는 모두 남와는 여모두가 용자로 로 보말에 그.
  「롬버드는 권을 갖요  성같 같은 없는 일어내 어왔다.
  「미은 일곱 지격는 목소리로 말했다.
  「내  뛰 오는 일곱 개발소리로 말했다.
  「그렇다면 다. 저사람이오.  그리나 판사는 여한 살격이 놀라 말이오.」
  그리는 그갑자까?」
   그의 낮
2446/2446 [==============================] - 16s 6ms/sample - loss: 0.0227
Epoch 94/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0398----Generating text after Epoch: 93 
---- diversity: 1.2
----Generating with seed: "놓
여 있었다.
  베러는 식탁 옆으로  갔다. 그녀 ""
놓
여 있었다.
  베러는 식탁 옆으로  갔다. 그녀 바보 있었다. 문그대로 움자에 그는 
주밀. 나갔다. 
을 「나 좀의 말했다.
  「틀자나도 틀림없이 우 들어떻 있었가에 우리의 모두 남갑 기다.

리는 모두 침서 도 사람이 뚜말이.
  「미소 렇소. 그러나 모요.」
  블로어가 말했다.
  「나도 세구에 여기 의자로 여자라 세요을 일곱 개개보았다.

  그리의 모습을 보라보았다. 암스트롱 우리가 해가 운는  만한 목소리로 말했다.
  「내  「틀람이오.」
   그리의 주습니다.」
  그들자 판사는 여라히 격도 세니다. 나

Epoch 105/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0140----Generating text after Epoch: 104 
---- diversity: 1.2
----Generating with seed: "  않으면 장군님이 오실 때까지  기다리겠
습니까?」
""
  않으면 장군님이 오실 때까지  기다리겠
습니까?」
  「나 위에밀리 브랜트에 몸하오.
우 비갔다. 테 「그렇다면 다. 암스트롱 의사는 가서에서 암스트롱 의사가 있습니다.」
  「…어 틀권쪽, 받지다. 암스트롱 」사  의사는 살이한 서 수 없는 권 가운데  거리 있었다. 태그리는 베러이 게 뛰오.
  그는 섬으로 오는 소 서는 거요.」
  「내 나는 의 가 없다. 
여 베러나도 없습니다.」
  판사는 그녀이바  앉아 여자에앗다.
  「내 나는 의 목로저스는 모두가들들 왔소. 베러가 성없는 살인범다.」
  베러의 가 없는 살를 열다. 태실다.」
  베러의 가 움는 일 수 있소.」
  베러가  입이 알 수 없는 살 해운 표 아론 것은 없지 않는 않았다.
  「오는 틀림없소. 우리들밖에 암스트롱 」사고 
작을 또요.」
  사로 얼굴을 문실한 로 수 없소.」
  그
2446/2446 [==============================] - 16s 7ms/sample - loss: 0.0141
Epoch 106/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0298----Generating text after Epoch: 105 
---- diversity: 1.2
----Generating with seed: "사는 헛기침을 하고 말했다.
  「유감스럽게도 우리는 ""
사는 헛기침을 하고 말했다.
  「유감스럽게도 우리는 모두 남이이 이제라의 으로 초보이 
거에 또이 기나 크지 않았다.
  「미플 세의에서 침자하  있었은 창 해서 기했다.

  등나 사람 의사로 여자살다.
  「내  세 사람이오. 암 수로저스 람만한  갖리고 목을 습로 바보 말했다.
  「내  세 사는 2단 있계요.」
  여롬버드는 곤차가운데  모고 있었다. 블로어 다른 
바에 의자이 오고 
바보이오. 암스트롱 의사는 알습니다.」
  판사는 있었다. 여자라로 도 아니다. 나도억다른면 판사는 얼굴을 문속르. 우리들 가
2446/2446 [==============================] - 19s 8ms/sample - loss: 0.0087
Epoch 117/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0039----Generating text after Epoch: 116 
---- diversity: 1.2
----Generating with seed: " 「매커서 장군이…….」
  「죽었군요!」
  베러의 ""
 「매커서 장군이…….」
  「죽었군요!」
  베러의 모시겠그와  였다. 
었다. 여자라 하나 여 있소. 그그
게 홀리고 목을 눈은 죄뱉고 있었다. 긴 침분을 보고 있었다. 그녀고 있었습니다.」
  「그렇습니다다.」
  「그러나 곧자로 말했다.
  「나는 의자리의 주니것도 우리가 해서 암스트롱 」
  블로어  등한 말했다.
  「나는 워그들의 모소리로 몹  「태진  아앉아 있었다. 비그리는 증거를 조용히 들어왔다.

  「내  처소. 판사는 냉하을 
 있는 일은 반금 」
  블로어  세시 이 한구가 거요. 그러나 거가 어요. 틀림없이 우리들밖에 
방 을 없지 않을  세요.

  「미면 여자라세 의자로 내려 세 있는 모소.」
  그리의 모습니다.」
  「나는 판사는 냉녀하니 다.」
  롬버드는 곤족의 목을 헐」
  긴나 있미이 일곱 
을 열었다.
  「오는 문을
2446/2446 [==============================] - 19s 8ms/sample - loss: 0.0039
Epoch 118/150
2432/2446 [

이 있느냐 없느냐 하는 거요. 혐러가 오. 그는 그의 생각하나  여자, 들리 모여자라, 목소리로 말했다.
  「내  두리머서에 몸다. 테이오는 계그다 
시 이미로 수 없소.  노장이 한 사람이오. 여 진격가을 길내리본다. 모두들가응 판사는 여자을 수냈거요. 우리들 가가 알 없는 격거요. 일곱오개 그는 어젯다. 저실는 테으로 내 만다리셨다. 모두들들 홀으리 오는 길을 열지 않는 니 태이에 일어요.」
   라감기다. 사정을 사람이 악나 이는 거요. 게 가  여자한 사람이오. 일  사람  모두 들의 목소리였다. 모두들렸 내 사람이오. 태그러나았다. 필요며 배발가 알 없의 목소리로 말했다.
  「내  세시겠
지 않지요?」
  의사가 냉는 
 접는 것은 아니다. 뒷리는 들사람을 뚜느새군보았오는 
실 이오. 뒷  증가운 로저스 의자로 말하다.
2446/2446 [==============================] - 16s 6ms/sample - loss: 0.0057
Epoch 129/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0167----Generating text after Epoch: 128 
---- diversity: 1.2
----Generating with seed: " 수 있습니
다.」
  「특히 심한 타격이 있었소?」
""
 수 있습니
다.」
  「특히 심한 타격이 있었소?」
  「없떻  오고 있었다. 
 트롱이나도 세. 그녀자라 온라 말했다.
  「나는 의자로 말했다.
  「나도가의자의 나서 그녀자, 말했다.
  「나도 세말 그소. 저스트, 와겠접나  여자, 하나 있었다.  그리로 말했다.
  「나도 세말에게 그들  흉가운  오는 권다. 모두들들 가운데 한 사람이오.」
  베러가 성스의 목소리였다. 그녀나바가 듯  때까기 그리고 햄었다. 비그는 베러의 목을 하지 않는데 식태가운 햄소.  그리가 해도 아니다. 나 「나도 의사는 
거을 일곱 개만한 목소리로 말했다.
  「

  「미니다. 여기분이오. 뒷내 그는 것 
2446/2446 [==============================] - 20s 8ms/sample - loss: 0.0232
Epoch 140/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0172----Generating text after Epoch: 139 
---- diversity: 1.2
----Generating with seed: "으로 들어갔다. 식당은 그들이 아까 나왔을 때 
그대로""
으로 들어갔다. 식당은 그들이 아까 나왔을 때 
그대로였다.」
  「그러나 판사는 얼을 없고 있었다.」
  「심감이 알 수 없지요. 우리들 가운데 악에밀이 꿰스신이 들어왔을 때

 않는 소.」
  베러가 성! 우리들 가운데 한 사람이오. 오 에밀리 브랜트가 뜨개질 하나 있었다.
  「미은 드릴이 꿰가 성격 한 목소리로 말했다.
  「그렇다. 그렇지더는 베러의 모을 보요. 서 수 없이 살녀이 면것가에…. 없다다.
  「내 습이야 세 에게 다. 
와 똑지 않는 것 없지 , 보이  판사는 여진진격격이 아요. 그러나 그가 어젯왔을 보요냐 에밀리 우리들 모두가서서한 사람이오. 그 이….」
  암스트롱이 떨며 말했다.
  「나서 세 에이 있력 게 가  모계트롱 」
  블로어가 말했다.
  「틀자로 자, 우리들 모는 거요. 또 오.」
  우리들 섬으로 오는 길을다.

 「
2446/2446 [==============================] - 16s 7ms/sample - loss: 0.0171
Epoch 141/150
2432/2446 [============================>.] - ETA: 0s - loss: 0.0118----Generating text after Epoch: 140 
---- diversity: 1.2
----Generating with seed: "
  에밀리 브랜트는 계속 뜨개질만  하고 있었다. 그